In [2]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import numpy as np
import pandas as pd
import warnings
from sklearn.exceptions import DataConversionWarning

# Suppress scikit-learn and other warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

2023-10-20 10:58:25.328559: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv("train.csv")
df['author'].fillna('Unknown', inplace=True)
df['title'].fillna('Ambiguous', inplace=True)
df['text'].fillna('Ambiguous', inplace=True)
df.drop_duplicates(inplace=True)

# TF-IDF Vectorization for text and title
vectorizer_text = TfidfVectorizer(max_features=5000,ngram_range=(1, 3))
X_text = vectorizer_text.fit_transform(df['text']).toarray()

vectorizer_title = TfidfVectorizer(max_features=1000,ngram_range=(1, 3))
X_title = vectorizer_title.fit_transform(df['title']).toarray()

# One-hot encoding for authors
encoder = OneHotEncoder()
X_author = encoder.fit_transform(df[['author']]).toarray()


In [5]:
# Train-test split
y = df['label'].values
X_train_text, X_test_text, X_train_title, X_test_title, X_train_author, X_test_author, y_train, y_test = train_test_split(
    X_text, X_title, X_author, y, test_size=0.2, random_state=42)

# Neural Network Architecture
input_text = Input(shape=(X_train_text.shape[1],))
input_title = Input(shape=(X_train_title.shape[1],))
input_author = Input(shape=(X_train_author.shape[1],))

# Layers for text
x1 = Dense(128, activation='relu')(input_text)
x1 = Dense(64, activation='relu')(x1)

# Layers for title
x2 = Dense(128, activation='relu')(input_title)
x2 = Dense(64, activation='relu')(x2)

# Layers for author
x3 = Dense(128, activation='relu')(input_author)
x3 = Dense(64, activation='relu')(x3)

# Concatenate
concat = Concatenate()([x1, x2, x3])

# Final layers
out = Dense(64, activation='relu')(concat)
out = Dense(1, activation='sigmoid')(out)

# Compile model
model = Model(inputs=[input_text, input_title, input_author], outputs=out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit([X_train_text, X_train_title, X_train_author], y_train, epochs=40, batch_size=32)

# Evaluate model
score = model.evaluate([X_test_text, X_test_title, X_test_author], y_test)
print(f"Test Accuracy: {score[1]}")

y_pred = model.predict([X_test_text, X_test_title, X_test_author])
y_pred = np.round(y_pred).flatten()  # Round the probabilities to get binary class labels

# Print classification report
print(classification_report(y_test, y_pred))

Epoch 1/40
520/520 [==============================] - 12s 19ms/step - loss: 0.0627 - accuracy: 0.9805
Epoch 2/40
520/520 [==============================] - 12s 23ms/step - loss: 8.1846e-04 - accuracy: 0.9998
Epoch 3/40
520/520 [==============================] - 11s 21ms/step - loss: 1.8488e-04 - accuracy: 0.9999
Epoch 4/40
520/520 [==============================] - 9s 17ms/step - loss: 2.5352e-05 - accuracy: 1.0000
Epoch 5/40
520/520 [==============================] - 9s 17ms/step - loss: 1.0767e-05 - accuracy: 1.0000
Epoch 6/40
520/520 [==============================] - 8s 16ms/step - loss: 5.7955e-06 - accuracy: 1.0000
Epoch 7/40
520/520 [==============================] - 8s 15ms/step - loss: 3.0126e-06 - accuracy: 1.0000
Epoch 8/40
520/520 [==============================] - 8s 15ms/step - loss: 1.7161e-06 - accuracy: 1.0000
Epoch 9/40
520/520 [==============================] - 7s 14ms/step - loss: 1.0595e-06 - accuracy: 1.0000
Epoch 10/40
520/520 [==============================] - 7

In [6]:
print(f"Test Accuracy: {score[1]}")

Test Accuracy: 0.998317301273346


In [26]:
def predict_article(title, text, author, model, vectorizer_text, vectorizer_title, encoder):
    # Transform title and text using the respective TfidfVectorizers
    X_title = vectorizer_title.transform([title]).toarray()
    X_text = vectorizer_text.transform([text]).toarray()

    # Transform the author using OneHotEncoder
    try:
        X_author = encoder.transform([[author]]).toarray()
    except:
        # If the author is not recognized from the training data
        X_author = np.zeros((1, len(encoder.categories_[0])))

    # Predict using the trained model
    prediction_prob = model.predict([X_text, X_title, X_author])
    
    # Return the binary prediction (0 for real news and 1 for fake news)
    return int(np.round(prediction_prob)[0][0])

# Example usage
title = "Aliens Land in Central Park!"
text = ("In a surprising turn of events, extraterrestrial beings made contact with Earth by landing their spaceship in New York's Central Park. "
       "Thousands of onlookers watched in awe as the unidentified creatures emerged, announcing their peaceful intentions. "
       "Authorities have quarantined the area and are in talks with the visitors.")
author = "Jacob"

prediction = predict_article(title, text, author, model, vectorizer_text, vectorizer_title, encoder)
print("Fake News" if prediction else "Real News")

1/1 [==============================] - 0s 356ms/step
Fake News


In [27]:
import numpy as np
import pandas as pd

def predict_all_articles(df, model, vectorizer_text, vectorizer_title, encoder):
    df['author'].fillna('Unknown', inplace=True)
    df['title'].fillna('Ambiguous', inplace=True)
    df['text'].fillna('Ambiguous', inplace=True)
    
    # Transform title and text for all articles
    X_title = vectorizer_title.transform(df['title']).toarray()
    X_text = vectorizer_text.transform(df['text']).toarray()

    # Transform the authors
    try:
        X_author = encoder.transform(df[['author']]).toarray()
    except:
        # If the author is not recognized from the training data
        X_author = np.zeros((df.shape[0], len(encoder.categories_[0])))
    
    # Make batch predictions
    prediction_probs = model.predict([X_text, X_title, X_author])
    
    # Round the probabilities to get binary class labels
    predictions = np.round(prediction_probs).flatten().astype(int)
    
    # Map 0 and 1 to "Real News" and "Fake News"
    prediction_labels = ["Real News" if p == 0 else "Fake News" for p in predictions]
    
    # Add a new column to the original DataFrame to store predictions
    df['Prediction'] = prediction_labels
    
    return df

def get_accuracy(df):
    # Map "Real News" and "Fake News" back to 0 and 1
    df['PredictionLabel'] = df['Prediction'].map({"Real News": 0, "Fake News": 1})
    
    # Calculate the number of correct predictions
    correct_predictions = df[df['label'] == df['PredictionLabel']].shape[0]
    
    # Calculate the total number of predictions
    total_predictions = df.shape[0]
    
    # Calculate the accuracy
    accuracy = (correct_predictions / total_predictions) * 100
    
    print(f"Accuracy: {accuracy}%")


In [28]:
df_test = pd.read_csv("test.csv")
df_submit = pd.read_csv("submit.csv")
# Combine test and submit datasets based on 'id'
df_test = pd.merge(df_test, df_submit, on='id')

df_with_predictions = predict_all_articles(df_test, model, vectorizer_text, vectorizer_title, encoder)
get_accuracy(df_with_predictions)


163/163 [==============================] - 1s 5ms/step
Accuracy: 67.75%


In [29]:
df_test_1 = pd.read_csv("Combined_modified.csv")
df_with_predictions_1 = predict_all_articles(df_test_1, model, vectorizer_text, vectorizer_title, encoder)
get_accuracy(df_with_predictions_1)

1404/1404 [==============================] - 6s 4ms/step
Accuracy: 52.360906944630045%
